In [2]:
from skimage.feature import hog
from skimage import io,color
from skimage.transform import resize
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn import metrics
import os
from  LBP_descriptor import LocalBinaryPatterns
import commonfunctions as cf
import cv2
import pandas as pd


# Get our training data 
X_train: features of training data.\
Y_train: labels of training data (1-->F, 0--> M).


In [3]:
Y_train=[]

files = os.listdir("Training_data/preprocessed/")

for file in files:
    if file[0]=='F':
        Y_train.append(1)
    else:
        Y_train.append(0)

Y_train=np.array(Y_train).astype(float)


# HOG feature

In [4]:
def HOG(img):
    img = np.array(resize(io.imread(img),(128,64))) 
    feature_vector, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(5,5), visualize=True)
    return feature_vector,hog_image


# LBP feature and histogram descriptor

In [5]:
# create object from LBP class to use it as our descriptor
# takes 2 parameters: number of data (train + test ) and number of neighbors
desc = LocalBinaryPatterns(258, 8)

# Extract features from training data

In [21]:
#  NOTE => to save your time: 
#  Run this cell once and the features will be saved in external file so that you can read them by running the next cell.
X_train=[]
HOG_train=[]
LBP_train=[]
files = os.listdir("Training_data/preprocessed/")

for file in files:
    # read the image
    img = io.imread("Training_data/preprocessed/"+file )

    # ------------------- HOG feature------------------------
    feature_vector,hog_image=HOG("Training_data/preprocessed/"+file)
    # --------------------------------------------------------
    HOG_train.append(feature_vector)
    
    #------------------- LBP feature------------------------
    
    img = cf.downSize(img , 0.5)
    hist = desc.describe(img)
    LBP_train.append(hist)
    #--------------------------------------------------------
    
    # concatenate all the features in X_train   
    feature_vector_temp=np.hstack((HOG_train,LBP_train)).tolist()
    X_train.append(feature_vector_temp[0])
    
    # reset them for the next img
    HOG_train=[]
    LBP_train=[]
    
#convert to numpy array
#X_train=np.array(X_train)

#write feature vector of each image in external file
with open('training_features.npy', 'wb') as f:
    np.save(f, X_train)
f.close()  
   

In [ ]:
# Read feature vector of train data from the npy file 
with open('training_features.npy', 'rb') as f:
    X_train = np.load(f,allow_pickle=True)
f.close() 

# Get our test data 

In [24]:
# NOTE => to save your time: 
# Run this cell once and the features will be saved in external files so that you can read them by running the next cell.
X_test=[]
Y_test=[]
HOG_test=[]
LBP_test=[]

files = os.listdir("Test_data/preprocessed/")

for file in files:
    if file[0]=='F':
        Y_test.append(1)
    else:
        Y_test.append(0)

    # read te img
    img = io.imread("Test_data/preprocessed/" +file )
        
    #------------------- HOG feature------------------------
    feature_vector,hog_image=HOG("Test_data/preprocessed/" +file)
    HOG_test.append(feature_vector)
    
    # #------------------- LBP feature------------------------
    img = cf.downSize(img , 0.5)
    hist = desc.describe(img)
    LBP_test.append(hist)
    #--------------------------------------------------------

    # concatenate all the features in X_train
    feature_test_temp=(np.hstack((HOG_test,LBP_test))).tolist()
    X_test.append( feature_test_temp[0] )

    # reset them for the next image
    HOG_test=[]
    LBP_test=[]

    
Y_test=np.array(Y_test)
#X_test=np.array(X_test)

#write feature vector of test data in external file
with open('test_features.npy', 'wb') as f:
    np.save(f, X_test)
f.close()   
#write labels of test data in external file
with open('Y_test.npy', 'wb') as f:
    np.save(f, Y_test)
f.close()     

In [ ]:
# Read feature vector of test data from the npy file 
with open('test_features.npy', 'rb') as f:
    X_test = np.load(f ,allow_pickle=True)
f.close()  
# Read labels of test data from the npy file 
with open('Y_test.npy', 'rb') as f:
    Y_test = np.load(f ,allow_pickle=True)
f.close()  

# Classification:

# 1. Random forest classifier

In [26]:

clf=RandomForestClassifier(n_estimators=1000)
clf.fit(X_train,Y_train)
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")


Accuracy: 57.692307692307686 %


# 2. Linear SVM classifier

In [25]:
clf=LinearSVC(C=100.0, random_state=42)
clf.fit(X_train,Y_train)
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")

Accuracy: 65.38461538461539 %


# 3. Adaboost classifier

In [27]:
clf=AdaBoostClassifier(n_estimators=500)
clf.fit(X_train,Y_train)
Y_Predicted=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_Predicted)*100,"%")

Accuracy: 57.692307692307686 %


# 4. KNN classifier

In [28]:
accuracies=[]
for k in range(1,30):
    clf=KNeighborsClassifier(n_neighbors = k)
    clf.fit(X_train,Y_train)
    Y_Predicted=clf.predict(X_test)
    accuracies.append(metrics.accuracy_score(Y_test, Y_Predicted)*100)

print("Accuracy:",accuracies[np.argmax(accuracies)],"%")

Accuracy: 63.46153846153846 %
